In [1]:
from evaluation.face_recognition_test import Face_Fecognition_test
from evaluation.ijb_evals import instantiate_list
from omegaconf import OmegaConf
from pathlib import Path
import numpy as np
from hydra.utils import instantiate

## Images with lowest SCF unc and highest proposed unc

In [2]:
cfg = OmegaConf.load("/app/configs/uncertainty_benchmark/ijb_evals.yaml")
method = cfg.open_set_identification_methods[0]
sampler = instantiate(method.sampler)
evaluation_function = instantiate(method.evaluation_function)
assert evaluation_function is not None
# if cfg.test_dataset.dataset_name == "survFace" and method.use_detector_score:
#     continue
template_pooling = instantiate(method.template_pooling_strategy)

In [3]:
open_set_identification_metrics = instantiate_list(cfg.open_set_identification_metrics)
if "open_set_uncertainty_metrics" in cfg:
    open_set_uncertainty_metrics = instantiate_list(cfg.open_set_uncertainty_metrics)
else:
    open_set_uncertainty_metrics = []
closed_set_identification_metrics = instantiate_list(
    cfg.closed_set_identification_metrics
)
verification_metrics = instantiate_list(cfg.verification_metrics)

test_dataset = instantiate(cfg.test_dataset)

verif_scores, verif_names = [], []
open_set_ident_scores, open_set_ident_names = [], []
closed_set_ident_scores, closed_set_ident_names = [], []

open_set_ident_rejection_scores, open_set_ident_rejection_names = [], []

# create result dirs:
dataset_name = cfg.test_dataset.dataset_name
open_set_identification_result_dir = (
    Path(cfg.exp_dir) / dataset_name / "open_set_identification"
)
open_set_identification_result_dir.mkdir(exist_ok=True, parents=True)

closed_set_identification_result_dir = (
    Path(cfg.exp_dir) / dataset_name / "closed_set_identification"
)
closed_set_identification_result_dir.mkdir(exist_ok=True, parents=True)

verification_result_dir = Path(cfg.exp_dir) / dataset_name / "verification"
verification_result_dir.mkdir(exist_ok=True, parents=True)

# create result tables place holders
open_set_recognition_result_metrics = {}
open_set_uncertainty_result_metrics = {}

open_set_ident_pretty_names = {}
closed_set_ident_pretty_names = {}
verication_pretty_names = {}
# define methods

tt = Face_Fecognition_test(
    sampler=sampler,
    evaluation_function=evaluation_function,
    test_dataset=test_dataset,
    embeddings_path=method.embeddings_path,
    template_pooling_strategy=template_pooling,
    use_detector_score=method.use_detector_score,
    use_two_galleries=cfg.use_two_galleries,
    recompute_template_pooling=cfg.recompute_template_pooling,
    open_set_identification_metrics=open_set_identification_metrics,
    closed_set_identification_metrics=closed_set_identification_metrics,
    verification_metrics=verification_metrics,
    open_set_uncertainty_metrics=open_set_uncertainty_metrics,
)

In [4]:
(
    g1_templates_feature,
    g1_template_unc,
    g1_unique_ids,
) = tt.get_template_subsets(tt.test_dataset.g1_templates, tt.test_dataset.g1_ids)
# print("g1_templates_feature:", g1_templates_feature.shape)  # (1772, 512)

(
    probe_templates_feature,
    probe_template_unc,
    probe_unique_ids,
) = tt.get_template_subsets(tt.test_dataset.probe_templates, tt.test_dataset.probe_ids)

# sample probe feature vectors

probe_templates_feature = tt.sampler(
    probe_templates_feature,
    probe_template_unc,
)

similarity, probe_score = tt.evaluation_function(
    probe_templates_feature,
    probe_template_unc,
    g1_templates_feature,
    g1_template_unc,
)

In [5]:
unc_scores = {}
for unc_metric in tt.open_set_uncertainty_metrics:
    if hasattr(unc_metric, "data_variance_weight") is False:
        continue
    if unc_metric.data_variance_weight == 0:
        unc_scores["probe_unc"] = unc_metric
    elif unc_metric.data_variance_weight == 1:
        unc_scores["scf_unc"] = unc_metric
    else:
        continue

In [6]:
probe_score = unc_scores["probe_unc"](
    probe_ids=probe_unique_ids,
    probe_template_unc=probe_template_unc,
    gallery_ids=g1_unique_ids,
    similarity=similarity,
    probe_score=probe_score,
)[1]

In [7]:
scf_score = unc_scores["scf_unc"](
    probe_ids=probe_unique_ids,
    probe_template_unc=probe_template_unc,
    gallery_ids=g1_unique_ids,
    similarity=similarity,
    probe_score=probe_score,
)[1]

In [8]:
most_conf_ids = np.argsort(scf_score)

In [9]:
unique_templates = np.unique(tt.test_dataset.probe_templates, return_index=False)

In [10]:
unique_templates[most_conf_ids]

array([ 15883,  17099, 187475, ...,  21742, 179072,  16673])

In [11]:
from evaluation.data_tools import read_meta_columns_to_int
import pandas as pd

data_path = Path(cfg.test_dataset.dataset_path)
media_list_path = data_path / "meta" / f"{dataset_name.lower()}_face_tid_mid.txt"
pair_list_path = data_path / "meta" / f"{dataset_name.lower()}_template_pair_label.txt"
img_path = data_path / "loose_crop"
img_list_path = data_path / "meta" / f"{dataset_name.lower()}_name_5pts_score.txt"

meta = pd.read_csv(media_list_path, sep=" ", skiprows=0, header=None).values

In [12]:
image_paths = meta[:, 0]
template_ids = meta[:, 1]

In [13]:
sorted_templates = unique_templates[most_conf_ids]

In [14]:
from shutil import copyfile

test_foulder = Path("outputs/test_high_scf")
test_foulder.mkdir(exist_ok=True)
for template_id in sorted_templates[:10]:
    test_template_path = test_foulder / str(template_id)
    test_template_path.mkdir(exist_ok=True)

    # copy image
    for image_name in image_paths[template_ids == template_id]:
        in_image_name = data_path / "loose_crop" / image_name
        out_image_name = test_template_path / image_name
        copyfile(in_image_name, out_image_name)

In [15]:
from shutil import copyfile

test_foulder = Path("outputs/test_low_scf")
test_foulder.mkdir(exist_ok=True)
for template_id in sorted_templates[-10:]:
    test_template_path = test_foulder / str(template_id)
    test_template_path.mkdir(exist_ok=True)

    # copy image
    for image_name in image_paths[template_ids == template_id]:
        in_image_name = data_path / "loose_crop" / image_name
        out_image_name = test_template_path / image_name
        copyfile(in_image_name, out_image_name)

### high scf low prob

In [16]:
similarity = similarity[:, 0, :]

In [17]:
def copy_template_images(data_path, test_template_path, template_id):
    test_template_path.mkdir(exist_ok=True)
    # copy template images
    for image_name in image_paths[template_ids == template_id]:
        in_image_name = data_path / "loose_crop" / image_name
        out_image_name = test_template_path / image_name
        copyfile(in_image_name, out_image_name)

In [18]:
from shutil import copyfile, rmtree

test_foulder = Path("outputs/test_high_scf_low_prob_neighbors_g1")
in_galley_path = test_foulder / "in_gallery_probe"
out_of_galley_path = test_foulder / "out_of_gallery_probe"
rmtree(test_foulder, ignore_errors=True)
test_foulder.mkdir(exist_ok=True)
g1_unique_templates = np.unique(tt.test_dataset.g1_templates, return_index=False)
scf_size = 2000

max_scf_prob_scores_ids = np.argsort(probe_score[most_conf_ids][:scf_size])[::-1]
prob_score_template_ids = unique_templates[most_conf_ids][:scf_size][
    max_scf_prob_scores_ids
]

scf_conf = scf_score[most_conf_ids][:scf_size][max_scf_prob_scores_ids]
probe_conf = probe_score[most_conf_ids][:scf_size][max_scf_prob_scores_ids]
for i, probe_template_id in enumerate(prob_score_template_ids):
    template_pos = np.where(unique_templates == probe_template_id)[0][0]
    is_in_gallery = np.isin(probe_unique_ids[template_pos], g1_unique_ids)
    most_similar_gallery_ids = np.argsort(similarity[template_pos, :])[::-1]
    most_similar_templates = g1_unique_templates[most_similar_gallery_ids[:4]]
    if is_in_gallery:
        out_path = in_galley_path
    else:
        out_path = out_of_galley_path
    if probe_conf[i] < 5.43e-13:
        continue
    probe_template_path = (
        out_path
        / f"probe_id-{str(probe_unique_ids[template_pos])}_scf-unc-{scf_conf[i]}_prob-unc-{probe_conf[i]}"
    )
    probe_template_path.mkdir(exist_ok=True, parents=True)
    copy_template_images(
        data_path, probe_template_path / "probe_images", probe_template_id
    )
    for j, template_id in enumerate(most_similar_templates):
        cos_sim = similarity[template_pos, most_similar_gallery_ids[j]]
        id = g1_unique_ids[most_similar_gallery_ids[j]]
        test_template_path = (
            probe_template_path / f"close-gallery-id-{str(id)}_sim-{cos_sim}"
        )
        test_template_path.mkdir(exist_ok=True, parents=True)
        copy_template_images(data_path, test_template_path, template_id)

In [19]:
similarity.shape

(19593, 1772)

In [20]:
most_similar_gallery_ids[j]

583